# Dynamic parameters

To facilitate experimenting around weighting, placeholder `Param`s can be utilised at query definition, which can be filled when running the query. 

In [1]:
%pip install superlinked==12.22.3

In [2]:
import pandas as pd
from superlinked import framework as sl

pd.set_option("display.max_colwidth", 100)

In [3]:
class Paragraph(sl.Schema):
    id: sl.IdField
    body: sl.String
    like_count: sl.Integer


paragraph = Paragraph()

body_space = sl.TextSimilaritySpace(
    text=paragraph.body, model="sentence-transformers/all-mpnet-base-v2"
)
like_space = sl.NumberSpace(
    number=paragraph.like_count, min_value=0, max_value=100, mode=sl.Mode.MAXIMUM
)
paragraph_index = sl.Index([body_space, like_space])

In [4]:
source: sl.InMemorySource = sl.InMemorySource(paragraph)
executor = sl.InMemoryExecutor(sources=[source], indices=[paragraph_index])
app = executor.run()

In [5]:
source.put(
    [
        {
            "id": "paragraph-1",
            "body": "Glorious animals live in the wilderness.",
            "like_count": 75,
        },
        {
            "id": "paragraph-2",
            "body": "Growing computation power enables advancements in AI.",
            "like_count": 10,
        },
    ]
)

## Using dynamic Params in the query

When defining the query, placeholder `Param`s can be used to indicate actual weights and other parameters will be filled when running the query.

In [6]:
query = (
    sl.Query(
        paragraph_index,
        weights={
            body_space: sl.Param("body_space_weight"),
            like_space: sl.Param(
                "like_space_weight", default=0.5
            ),  # Params' default value can be overriden.
        },
    )
    .find(paragraph)
    .similar(body_space, sl.Param("query_text"))
)

`Param`s generally have default values. The general guiding principle is that the Param will not affect the query results unless explicitly specified to do so. To list the most important examples: 
- space weights (defined in the `weights` argument of a `Query`) default to `0.0` (unless overriden using the `default` argument of `Param`)
- `.similar` and `.with_vector` clause inputs default to being empty, effectively getting removed from the `Query`
- `.similar` and `.with_vector` clause weight `Param`s default to `1.0` (but with an empty default input they take no effect unless specified)

In [7]:
body_based_result = app.query(
    query,
    query_text="How computation power changed the course of AI?",
    body_space_weight=1,
    like_space_weight=0,
)

body_based_result.to_pandas()

,body,like_count,id,similarity_score,rank
0,Growing computation power enables advancements in AI.,10,paragraph-2,0.454417,0
1,Glorious animals live in the wilderness.,75,paragraph-1,-0.012840,1


In [8]:
like_based_result = app.query(
    query,
    query_text="How computation power changed the course of AI?",
    body_space_weight=0,
    like_space_weight=1,
)

like_based_result.to_pandas()

,body,like_count,id,similarity_score,rank
0,Glorious animals live in the wilderness.,75,paragraph-1,0.461940,0
1,Growing computation power enables advancements in AI.,10,paragraph-2,0.078217,1


This feature enables reusing the query for different setting by simply supplying it with alternate weightings.